Histogram-Based Gradient Boosting Classifier

In [ ]:
import numpy as np


class Node:
    def __init__(self, prediction=None, feature_index=None, threshold=None, left=None, right=None):
        self.prediction = prediction
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right


class HistogramTree:
    def __init__(self, max_depth=3, min_samples_split=2, n_bins=256):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_bins = n_bins
        self.root = None
        self.bins = None


    def _bin_data(self, X):
        """Bin continuous data into discrete buckets for faster processing."""
        self.bins = np.linspace(np.min(X, axis=0), np.max(X, axis=0), self.n_bins)
        X_binned = np.digitize(X, bins=self.bins) - 1  # Bin indices for each feature
        return X_binned


    def _calculate_leaf_value(self, gradients):
        """Calculate the prediction for a leaf node."""
        return np.mean(gradients)


    def _best_split(self, X_binned, gradients):
        """Find the best split by iterating through all features and thresholds."""
        best_gain = -float("inf")
        best_split = {"feature_index": None, "threshold": None}


        for feature_index in range(X_binned.shape[1]):
            unique_thresholds = np.unique(X_binned[:, feature_index])
            for threshold in unique_thresholds:
                left_indices = X_binned[:, feature_index] <= threshold
                right_indices = X_binned[:, feature_index] > threshold


                if sum(left_indices) < self.min_samples_split or sum(right_indices) < self.min_samples_split:
                    continue


                left_gradient = gradients[left_indices]
                right_gradient = gradients[right_indices]


                gain = self._calculate_gain(left_gradient, right_gradient)


                if gain > best_gain:
                    best_gain = gain
                    best_split["feature_index"] = feature_index
                    best_split["threshold"] = threshold


        return best_split if best_gain > 0 else None


    def _calculate_gain(self, left_gradient, right_gradient):
        """Calculate gain based on reduction in variance."""
        var_total = np.var(np.concatenate([left_gradient, right_gradient]))
        var_left = np.var(left_gradient)
        var_right = np.var(right_gradient)


        weight_left = len(left_gradient) / (len(left_gradient) + len(right_gradient))
        weight_right = 1 - weight_left


        gain = var_total - (weight_left * var_left + weight_right * var_right)
        return gain


    def _build_tree(self, X_binned, gradients, depth):
        """Recursively build the tree based on histogram binning and gain calculation."""
        if depth == self.max_depth or len(X_binned) < self.min_samples_split:
            prediction = self._calculate_leaf_value(gradients)
            return Node(prediction=prediction)


        split = self._best_split(X_binned, gradients)
        if split is None:
            prediction = self._calculate_leaf_value(gradients)
            return Node(prediction=prediction)


        feature_index, threshold = split["feature_index"], split["threshold"]
        left_indices = X_binned[:, feature_index] <= threshold
        right_indices = ~left_indices


        left_child = self._build_tree(X_binned[left_indices], gradients[left_indices], depth + 1)
        right_child = self._build_tree(X_binned[right_indices], gradients[right_indices], depth + 1)


        return Node(feature_index=feature_index, threshold=threshold, left=left_child, right=right_child)


    def fit(self, X, gradients):
        X_binned = self._bin_data(X)
        self.root = self._build_tree(X_binned, gradients, 0)


    def _predict_row(self, x):
        node = self.root
        while node.left or node.right:
            if x[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.prediction


    def predict(self, X):
        X_binned = np.digitize(X, bins=self.bins) - 1  # Use the trained binning
        predictions = np.array([self._predict_row(row) for row in X_binned])
        return predictions


class HistGradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2, n_bins=256):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_bins = n_bins
        self.trees = []


    def _initialize_predictions(self, y):
        """Initialize predictions to log-odds for binary classification."""
        p = np.clip(np.mean(y), 1e-5, 1 - 1e-5)
        return np.log(p / (1 - p)) * np.ones(len(y))


    def _compute_gradients(self, y, pred):
        """Compute the gradients for binary cross-entropy loss."""
        probs = 1 / (1 + np.exp(-pred))  # Sigmoid for probability
        return y - probs


    def fit(self, X, y):
        pred = self._initialize_predictions(y)
        for _ in range(self.n_estimators):
            gradients = self._compute_gradients(y, pred)
            tree = HistogramTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, n_bins=self.n_bins)
            tree.fit(X, gradients)
            update = tree.predict(X)
            pred += self.learning_rate * update
            self.trees.append(tree)


    def predict_proba(self, X):
        """Predict probabilities using the boosted ensemble of trees."""
        pred = self._initialize_predictions(np.zeros(X.shape[0]))
        for tree in self.trees:
            pred += self.learning_rate * tree.predict(X)
        probs = 1 / (1 + np.exp(-pred))
        return np.vstack((1 - probs, probs)).T


    def predict(self, X):
        """Predict binary class based on the predicted probabilities."""
        probs = self.predict_proba(X)[:, 1]
        return (probs >= 0.5).astype(int)


# Example usage:
# X = np.random.rand(100, 1)  # Sample feature data
# y = np.random.randint(0, 2, 100)  # Sample binary targets
# model = HistGradientBoostingClassifier(n_estimators=10, max_depth=3)
# model.fit(X, y)
# predictions = model.predict(X)




AdaBoostClassifier

In [ ]:
import numpy as np


class DecisionStump:
    """
    A simple decision stump (a one-level decision tree) used as the weak classifier in AdaBoost.
    It finds the best feature and threshold to minimize classification error.
    """
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.polarity = 1  # Decides if feature values below or above the threshold belong to class 1
        self.alpha = None  # Weight of the stump in final prediction


    def fit(self, X, y, sample_weights):
        """
        Train the stump using weighted training samples.


        Parameters:
        X (np.array): Feature matrix
        y (np.array): Target labels
        sample_weights (np.array): Weights for each sample
        """
        n_samples, n_features = X.shape
        min_error = float('inf')


        # Loop over all features and find the best threshold for each
        for feature_i in range(n_features):
            feature_values = X[:, feature_i]
            unique_values = np.unique(feature_values)


            # Try each unique value in the feature as a threshold
            for threshold in unique_values:
                # Predict all samples as 1 or -1 based on the polarity
                for polarity in [1, -1]:
                    predictions = np.ones(n_samples)  # Initial predictions of 1
                    predictions[feature_values < threshold] = -1 if polarity == 1 else 1


                    # Calculate the error with the given polarity and threshold
                    misclassified = predictions != y
                    weighted_error = np.sum(sample_weights[misclassified])


                    # Keep track of the best threshold and polarity
                    if weighted_error < min_error:
                        min_error = weighted_error
                        self.polarity = polarity
                        self.threshold = threshold
                        self.feature_index = feature_i


    def predict(self, X):
        """
        Predict the class for each sample in X based on the threshold and polarity.


        Parameters:
        X (np.array): Feature matrix


        Returns:
        np.array: Predictions (-1 or 1)
        """
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)
        feature_values = X[:, self.feature_index]


        # Apply polarity to determine which side of the threshold is class -1
        if self.polarity == 1:
            predictions[feature_values < self.threshold] = -1
        else:
            predictions[feature_values >= self.threshold] = -1


        return predictions




class AdaBoostClassifier:
    """
    AdaBoost classifier that combines several decision stumps.
    """
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.stumps = []
        self.alphas = []


    def fit(self, X, y):
        """
        Train the AdaBoost model.


        Parameters:
        X (np.array): Feature matrix
        y (np.array): Target labels (assumed to be -1 and 1 for binary classification)
        """
        n_samples, _ = X.shape
        # Initialize weights for each sample
        sample_weights = np.ones(n_samples) / n_samples


        for _ in range(self.n_estimators):
            # Create a stump and train it
            stump = DecisionStump()
            stump.fit(X, y, sample_weights)
            predictions = stump.predict(X)


            # Calculate error and stump weight
            error = np.sum(sample_weights[predictions != y])
            if error > 0.5:
                continue  # Skip if error is above 0.5; this stump is too weak
            alpha = 0.5 * np.log((1.0 - error) / (error + 1e-10))  # Stump's weight in final classifier
            stump.alpha = alpha  # Store weight for this stump


            # Update weights for the samples
            sample_weights *= np.exp(-alpha * y * predictions)
            sample_weights /= np.sum(sample_weights)  # Normalize to sum to 1


            # Save this stump and its weight
            self.stumps.append(stump)
            self.alphas.append(alpha)


    def predict(self, X):
        """
        Predict the class labels for samples in X.


        Parameters:
        X (np.array): Feature matrix


        Returns:
        np.array: Predicted labels (-1 or 1)
        """
        # Weighted sum of predictions from each stump
        stump_preds = np.array([stump.alpha * stump.predict(X) for stump in self.stumps])
        y_pred = np.sum(stump_preds, axis=0)
        return np.sign(y_pred)  # Return the sign as the predicted class (-1 or 1)




# Example usage:
# Create a sample dataset
X = np.array([[1, 2], [2, 3], [3, 1], [4, 3], [5, 5]])
y = np.array([1, 1, -1, -1, 1])


# Convert labels from (0, 1) to (-1, 1) if necessary
y = np.where(y == 0, -1, y)


# Initialize and train AdaBoost
model = AdaBoostClassifier(n_estimators=10)
model.fit(X, y)


# Make predictions
predictions = model.predict(X)
print("Predictions:", predictions)




Predictions: [ 1.  1. -1.  1.  1.]


GradientBoostingCLassifier


In [ ]:
import numpy as np


class DecisionStump:
    """
    A simple decision stump (one-level decision tree) used as the weak learner in Gradient Boosting.
    It splits on a single feature with a threshold and predicts a constant value for each split.
    """
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.left_value = None
        self.right_value = None


    def fit(self, X, residuals):
        """
        Fit the stump on the data to minimize residual error.

        Parameters:
        X (np.array): Feature matrix
        residuals (np.array): Residuals (errors) of predictions from previous models
        """
        n_samples, n_features = X.shape
        min_error = float('inf')


        # Loop over each feature and threshold to find the best split
        for feature_i in range(n_features):
            feature_values = X[:, feature_i]
            unique_values = np.unique(feature_values)


            for threshold in unique_values:
                left_indices = feature_values < threshold
                right_indices = feature_values >= threshold


                # Calculate mean residuals in left and right regions
                left_value = np.mean(residuals[left_indices]) if np.sum(left_indices) > 0 else 0
                right_value = np.mean(residuals[right_indices]) if np.sum(right_indices) > 0 else 0


                # Calculate the residual sum of squares error for this split
                error = (
                    np.sum((residuals[left_indices] - left_value) ** 2) +
                    np.sum((residuals[right_indices] - right_value) ** 2)
                )


                # Update the stump parameters if this split is better
                if error < min_error:
                    min_error = error
                    self.feature_index = feature_i
                    self.threshold = threshold
                    self.left_value = left_value
                    self.right_value = right_value


    def predict(self, X):
        """
        Predict the residuals for each sample in X based on the threshold split.

        Parameters:
        X (np.array): Feature matrix


        Returns:
        np.array: Predicted residuals
        """
        feature_values = X[:, self.feature_index]
        predictions = np.where(feature_values < self.threshold, self.left_value, self.right_value)
        return predictions




class GradientBoostingClassifier:
    """
    Gradient Boosting Classifier that combines weak learners to minimize classification error.
    """
    def __init__(self, n_estimators=50, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.stumps = []
        self.stump_weights = []


    def fit(self, X, y):
        """
        Train the Gradient Boosting model on the training data.


        Parameters:
        X (np.array): Feature matrix
        y (np.array): Target labels (assumed to be -1 and 1 for binary classification)
        """
        # Initialize predictions to zero
        y_pred = np.zeros(len(y))


        for _ in range(self.n_estimators):
            # Calculate residuals
            residuals = y - y_pred


            # Train a new decision stump on the residuals
            stump = DecisionStump()
            stump.fit(X, residuals)
            stump_predictions = stump.predict(X)


            # Calculate stump weight and update predictions
            # Note: For a classification task, we multiply by learning_rate to control step size
            y_pred += self.learning_rate * stump_predictions


            # Save the stump
            self.stumps.append(stump)
            self.stump_weights.append(self.learning_rate)


    def predict(self, X):
        """
        Predict the class labels for samples in X.


        Parameters:
        X (np.array): Feature matrix


        Returns:
        np.array: Predicted labels (-1 or 1)
        """
        # Initialize predictions to zero
        y_pred = np.zeros(X.shape[0])


        # Aggregate predictions from each stump
        for stump, weight in zip(self.stumps, self.stump_weights):
            y_pred += weight * stump.predict(X)


        # Return final prediction
        return np.sign(y_pred)




# Example usage:
# Create a sample dataset
X = np.array([[1, 2], [2, 3], [3, 1], [4, 3], [5, 5]])
y = np.array([1, 1, -1, -1, 1])


# Convert labels from (0, 1) to (-1, 1) if necessary
y = np.where(y == 0, -1, y)


# Initialize and train Gradient Boosting
model = GradientBoostingClassifier(n_estimators=10, learning_rate=0.1)
model.fit(X, y)


# Make predictions
predictions = model.predict(X)
print("Predictions:", predictions)

Random Forest Classifier

In [ ]:
import numpy as np
from collections import Counter
import random


class DecisionTree:
    """
    A simple decision tree classifier used as a base learner in the random forest.
    """
    def __init__(self, max_depth=None, min_samples_split=2, n_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.tree = None


    def fit(self, X, y):
        """
        Build a decision tree by recursively splitting the data based on the best feature.
        """
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.tree = self._grow_tree(X, y)


    def _grow_tree(self, X, y, depth=0):
        """
        Recursively grow the tree by finding the best split.
        """
        n_samples, n_features = X.shape
        num_class_labels = len(np.unique(y))


        # Stop conditions
        if depth >= self.max_depth or num_class_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}


        # Randomly select features to consider for splitting
        feature_indices = np.random.choice(n_features, self.n_features, replace=False)


        # Find the best split
        best_feature, best_threshold = self._best_split(X, y, feature_indices)
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return {"leaf": True, "value": leaf_value}


        # Split the dataset
        left_indices = X[:, best_feature] < best_threshold
        right_indices = ~left_indices
        left_subtree = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
        return {"leaf": False, "feature": best_feature, "threshold": best_threshold, "left": left_subtree, "right": right_subtree}


    def _best_split(self, X, y, feature_indices):
        """
        Find the best feature and threshold to split on based on Gini impurity.
        """
        best_gini = float("inf")
        split = {"feature": None, "threshold": None}


        for feature_index in feature_indices:
            X_column = X[:, feature_index]
            thresholds = np.unique(X_column)


            for threshold in thresholds:
                left_indices = X_column < threshold
                right_indices = ~left_indices


                if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                    continue


                gini = self._gini(y[left_indices], y[right_indices])


                if gini < best_gini:
                    best_gini = gini
                    split["feature"] = feature_index
                    split["threshold"] = threshold


        return split["feature"], split["threshold"]


    def _gini(self, left_labels, right_labels):
        """
        Calculate the Gini impurity of a split.
        """
        def gini_impurity(labels):
            classes, counts = np.unique(labels, return_counts=True)
            probs = counts / len(labels)
            return 1 - np.sum(probs ** 2)


        n_left, n_right = len(left_labels), len(right_labels)
        n_total = n_left + n_right


        gini_left = gini_impurity(left_labels)
        gini_right = gini_impurity(right_labels)


        return (n_left / n_total) * gini_left + (n_right / n_total) * gini_right


    def _most_common_label(self, y):
        """
        Find the most common label in a set of labels.
        """
        counter = Counter(y)
        return counter.most_common(1)[0][0]


    def predict(self, X):
        """
        Predict the class label for each sample in X.
        """
        return np.array([self._predict_sample(x, self.tree) for x in X])


    def _predict_sample(self, x, tree):
        """
        Recursively traverse the tree to predict the label for a single sample.
        """
        if tree["leaf"]:
            return tree["value"]


        feature_value = x[tree["feature"]]
        if feature_value < tree["threshold"]:
            return self._predict_sample(x, tree["left"])
        else:
            return self._predict_sample(x, tree["right"])




class RandomForestClassifier:
    """
    Random Forest Classifier that aggregates predictions from multiple decision trees.
    """
    def __init__(self, n_estimators=100, max_depth=None, min_samples_split=2, n_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []


    def fit(self, X, y):
        """
        Train the random forest by training each decision tree on a bootstrap sample.
        """
        self.trees = []
        for _ in range(self.n_estimators):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, n_features=self.n_features)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)


    def _bootstrap_sample(self, X, y):
        """
        Generate a bootstrap sample from the dataset (sampling with replacement).
        """
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]


    def predict(self, X):
        """
        Predict the class labels by taking a majority vote across all trees.
        """
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return self._majority_vote(tree_preds)


    def _majority_vote(self, predictions):
        """
        Perform a majority vote to determine the final predicted class.
        """
        return np.array([Counter(preds).most_common(1)[0][0] for preds in predictions.T])




# Example usage:
# Create a sample dataset
X = np.array([[1, 2], [2, 3], [3, 1], [4, 3], [5, 5]])
y = np.array([0, 0, 1, 1, 0])


# Initialize and train the random forest
model = RandomForestClassifier(n_estimators=10, max_depth=3)
model.fit(X, y)


# Make predictions
predictions = model.predict(X)
print("Predictions:", predictions)

BernoulliNB classifier:

In [ ]:
import numpy as np


class BernoulliNB:
    def __init__(self, alpha=1.0):
        self.alpha = alpha


    def fit(self, X, y):
        self.classes, class_counts = np.unique(y, return_counts=True)
        self.class_priors = class_counts / len(y)


        self.feature_probs = {}


        for cls in self.classes:
            X_cls = X[y == cls]

            feature_prob = (X_cls.sum(axis=0) + self.alpha) / (X_cls.shape[0] + 2 * self.alpha)

            self.feature_probs[cls] = feature_prob


    def predict_log_proba(self, X):
        log_probs = []


        for cls in self.classes:
            log_prior = np.log(self.class_priors[np.where(self.classes == cls)][0])

            log_likelihood = X * np.log(self.feature_probs[cls]) + (1 - X) * np.log(1 - self.feature_probs[cls])
            total_log_prob = log_prior + log_likelihood.sum(axis=1)


            log_probs.append(total_log_prob)


        return np.array(log_probs).T


    def predict(self, X):
        log_probs = self.predict_log_proba(X)
        return self.classes[np.argmax(log_probs, axis=1)]




# Sample binary dataset
X = np.array([[1, 0, 1], [0, 1, 0], [1, 1, 1], [0, 0, 0]])
y = np.array([1, 0, 1, 0])


model = BernoulliNB(alpha=1.0)
model.fit(X, y)


predictions = model.predict(X)
print("Predictions:", predictions)




Calibrated Classifier CV:

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier


class CalibratedClassifierCV:
    def __init__(self, base_estimator, method='sigmoid', cv=5):
        self.base_estimator = base_estimator
        self.method = method
        self.cv = cv
        self.calibrators = []


    def fit(self, X, y):
        kf = KFold(n_splits=self.cv)

        oof_preds = np.zeros(len(y))
        oof_labels = np.zeros(len(y))


        for train_idx, calibrate_idx in kf.split(X):
            X_train, y_train = X[train_idx], y[train_idx]
            X_calibrate, y_calibrate = X[calibrate_idx], y[calibrate_idx]

            estimator = self._clone_base_estimator()
            estimator.fit(X_train, y_train)


            probas = estimator.predict_proba(X_calibrate)[:, 1]


            oof_preds[calibrate_idx] = probas
            oof_labels[calibrate_idx] = y_calibrate

            if self.method == 'sigmoid':
                calibrator = LogisticRegression()
                calibrator.fit(probas.reshape(-1, 1), y_calibrate)
                self.calibrators.append(calibrator)
            else:
                raise ValueError("Currently, only 'sigmoid' (Platt scaling) is implemented.")


        self.base_estimator.fit(X, y)


    def predict_proba(self, X):
        probas = self.base_estimator.predict_proba(X)[:, 1]
        calibrated_probas = np.zeros(len(probas))


        for calibrator in self.calibrators:
            calibrated_probas += calibrator.predict_proba(probas.reshape(-1, 1))[:, 1]

        calibrated_probas /= len(self.calibrators)
        return np.vstack([1 - calibrated_probas, calibrated_probas]).T


    def _clone_base_estimator(self):
        return type(self.base_estimator)(**self.base_estimator.get_params())


    def predict(self, X):
        calibrated_probas = self.predict_proba(X)
        return (calibrated_probas[:, 1] >= 0.5).astype(int)




# Generate sample data
X, y = make_classification(n_samples=500, n_features=10, random_state=42)


base_estimator = RandomForestClassifier(n_estimators=10, random_state=42)
calibrated_clf = CalibratedClassifierCV(base_estimator, method='sigmoid', cv=5)


calibrated_clf.fit(X, y)


calibrated_probs = calibrated_clf.predict_proba(X)
calibrated_preds = calibrated_clf.predict(X)


print("Calibrated Probabilities:", calibrated_probs[:5])
print("Calibrated Predictions:", calibrated_preds[:5])

Logistic Regression CV:

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification


class LogisticRegressionCV:
    def __init__(self, Cs=None, cv=5, solver='lbfgs', max_iter=100):
        if Cs is None:
            Cs = [0.01, 0.1, 1.0, 10.0, 100.0]
        self.Cs = Cs
        self.cv = cv
        self.solver = solver
        self.max_iter = max_iter
        self.best_C = None
        self.best_model = None


    def fit(self, X, y):
        kf = KFold(n_splits=self.cv)
        best_score = -np.inf


        for C in self.Cs:
            scores = []


            for train_idx, test_idx in kf.split(X):
                X_train, X_test = X[train_idx], X[test_idx]
                y_train, y_test = y[train_idx], y[test_idx]


                model = LogisticRegression(C=C, solver=self.solver, max_iter=self.max_iter)
                model.fit(X_train, y_train)


                score = model.score(X_test, y_test)
                scores.append(score)


            mean_score = np.mean(scores)


            if mean_score > best_score:
                best_score = mean_score
                self.best_C = C
                self.best_model = model


    def predict(self, X):
        if self.best_model is None:
            raise Exception("You must fit the model before predicting.")
        return self.best_model.predict(X)


    def predict_proba(self, X):
        if self.best_model is None:
            raise Exception("You must fit the model before predicting.")
        return self.best_model.predict_proba(X)


    def get_best_params(self):
        return self.best_C


# Generate sample data
X, y = make_classification(n_samples=500, n_features=10, random_state=42)


logistic_cv = LogisticRegressionCV(Cs=[0.01, 0.1, 1.0, 10.0, 100.0], cv=5)
logistic_cv.fit(X, y)


predicted_probs = logistic_cv.predict_proba(X)
predicted_labels = logistic_cv.predict(X)


print("Best C:", logistic_cv.get_best_params())
print("Predicted Probabilities (first 5):", predicted_probs[:5])
print("Predicted Labels (first 5):", predicted_labels[:5])

Logistic Regression:

In [ ]:
import numpy as np
from sklearn.datasets import make_classification


class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, tolerance=1e-6):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.weights = None
        self.bias = None


    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))


    def fit(self, X, y):
        n_samples, n_features = X.shape

        self.weights = np.zeros(n_features)
        self.bias = 0


        for _ in range(self.max_iter):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)


            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)


            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db


            if np.linalg.norm(dw) < self.tolerance and abs(db) < self.tolerance:
                break


    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return (y_predicted >= 0.5).astype(int)


    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        return np.vstack([1 - y_predicted, y_predicted]).T


    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions == y)


# Generate sample data
X, y = make_classification(n_samples=500, n_features=10, random_state=42)


logistic_regression = LogisticRegression(learning_rate=0.01, max_iter=1000)
logistic_regression.fit(X, y)


predicted_probs = logistic_regression.predict_proba(X)
predicted_labels = logistic_regression.predict(X)


accuracy = logistic_regression.score(X, y)


print("Predicted Probabilities (first 5):", predicted_probs[:5])
print("Predicted Labels (first 5):", predicted_labels[:5])
print("Model Accuracy:", accuracy)

MLPClassifier

In [ ]:
import numpy as np


# Define activation functions and their derivatives
def relu(x):
    return np.maximum(0, x)


def relu_derivative(x):
    return (x > 0).astype(float)


def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)


def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = -np.log(y_pred[range(n_samples), y_true])
    loss = np.sum(logp) / n_samples
    return loss


def cross_entropy_derivative(y_true, y_pred):
    n_samples = y_true.shape[0]
    grad = y_pred
    grad[range(n_samples), y_true] -= 1
    grad = grad / n_samples
    return grad


class MLPClassifier:
    """
    Multilayer Perceptron (MLP) Classifier with a single hidden layer.
    """
    def __init__(self, n_inputs, n_hidden, n_outputs, learning_rate=0.01, epochs=1000):
        # Network architecture
        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_outputs = n_outputs
        self.learning_rate = learning_rate
        self.epochs = epochs

        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.n_inputs, self.n_hidden) * 0.01
        self.bias_hidden = np.zeros((1, self.n_hidden))
        self.weights_hidden_output = np.random.randn(self.n_hidden, self.n_outputs) * 0.01
        self.bias_output = np.zeros((1, self.n_outputs))


    def forward(self, X):
        """
        Forward propagation through the network.
        """
        # Input to hidden layer
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = relu(self.hidden_input)


        # Hidden to output layer
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.output = softmax(self.output_input)
        return self.output


    def backward(self, X, y):
        """
        Backward propagation to update weights and biases.
        """
        n_samples = X.shape[0]

        # Compute gradient for output layer
        output_error = cross_entropy_derivative(y, self.output)
        d_weights_hidden_output = np.dot(self.hidden_output.T, output_error)
        d_bias_output = np.sum(output_error, axis=0, keepdims=True)


        # Backpropagate error to hidden layer
        hidden_error = np.dot(output_error, self.weights_hidden_output.T) * relu_derivative(self.hidden_input)
        d_weights_input_hidden = np.dot(X.T, hidden_error)
        d_bias_hidden = np.sum(hidden_error, axis=0, keepdims=True)


        # Update weights and biases
        self.weights_hidden_output -= self.learning_rate * d_weights_hidden_output
        self.bias_output -= self.learning_rate * d_bias_output
        self.weights_input_hidden -= self.learning_rate * d_weights_input_hidden
        self.bias_hidden -= self.learning_rate * d_bias_hidden


    def fit(self, X, y):
        """
        Train the MLP using forward and backward propagation.
        """
        for epoch in range(self.epochs):
            # Forward pass
            predictions = self.forward(X)


            # Loss calculation
            loss = cross_entropy_loss(y, predictions)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}/{self.epochs}, Loss: {loss}")


            # Backward pass
            self.backward(X, y)


    def predict(self, X):
        """
        Predict class labels for samples in X.
        """
        output_probs = self.forward(X)
        return np.argmax(output_probs, axis=1)




# Example usage:
# Create a sample dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])  # XOR problem


# Initialize MLP Classifier with 2 input neurons, 3 hidden neurons, and 2 output neurons
mlp = MLPClassifier(n_inputs=2, n_hidden=3, n_outputs=2, learning_rate=0.1, epochs=1000)


# Convert labels to one-hot encoding for softmax
y_onehot = np.zeros((y.size, 2))
y_onehot[np.arange(y.size), y] = 1


# Train the model
mlp.fit(X, y)


# Make predictions
predictions = mlp.predict(X)
print("Predictions:", predictions)

LinearDiscriminantAnalysis

In [ ]:
import numpy as np


class LinearDiscriminantAnalysis:
    """
    Linear Discriminant Analysis (LDA) Classifier
    """
    def __init__(self, n_components=None):
        self.n_components = n_components
        self.means_ = None
        self.scalings_ = None


    def fit(self, X, y):
        """
        Fit LDA model to training data.

        Parameters:
        - X: Feature matrix, shape (n_samples, n_features)
        - y: Target labels, shape (n_samples,)
        """
        n_features = X.shape[1]
        class_labels = np.unique(y)

        # Calculate the mean vectors for each class
        mean_vectors = []
        for c in class_labels:
            mean_vectors.append(np.mean(X[y == c], axis=0))
        overall_mean = np.mean(X, axis=0)


        # Initialize within-class and between-class scatter matrices
        S_W = np.zeros((n_features, n_features))
        S_B = np.zeros((n_features, n_features))


        # Compute within-class scatter matrix S_W
        for i, c in enumerate(class_labels):
            class_scatter = np.cov(X[y == c].T) * (X[y == c].shape[0] - 1)
            S_W += class_scatter


        # Compute between-class scatter matrix S_B
        for i, mean_vec in enumerate(mean_vectors):
            n = X[y == class_labels[i]].shape[0]
            mean_diff = (mean_vec - overall_mean).reshape(n_features, 1)
            S_B += n * (mean_diff).dot(mean_diff.T)


        # Solve the generalized eigenvalue problem for S_W^-1 * S_B
        eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))


        # Sort eigenvectors by descending eigenvalues
        sorted_indices = np.argsort(eigen_values)[::-1]
        eigen_values = eigen_values[sorted_indices]
        eigen_vectors = eigen_vectors[:, sorted_indices]


        # Select the top n_components eigenvectors
        if self.n_components:
            eigen_vectors = eigen_vectors[:, :self.n_components]

        self.scalings_ = eigen_vectors


    def transform(self, X):
        """
        Project data onto the LDA components.


        Parameters:
        - X: Feature matrix, shape (n_samples, n_features)


        Returns:
        - X_lda: Transformed feature matrix, shape (n_samples, n_components)
        """
        return np.dot(X, self.scalings_)


    def fit_transform(self, X, y):
        """
        Fit the model and then transform X to LDA components.


        Parameters:
        - X: Feature matrix, shape (n_samples, n_features)
        - y: Target labels, shape (n_samples,)


        Returns:
        - X_lda: Transformed feature matrix, shape (n_samples, n_components)
        """
        self.fit(X, y)
        return self.transform(X)


    def predict(self, X):
        """
        Predict class labels for samples in X.


        Parameters:
        - X: Feature matrix, shape (n_samples, n_features)


        Returns:
        - y_pred: Predicted labels, shape (n_samples,)
        """
        X_lda = self.transform(X)
        distances = []
        for x in X_lda:
            distances.append([np.linalg.norm(x - mean) for mean in self.means_])
        return np.argmin(distances, axis=1)


# Example usage:
# Create a sample dataset
X = np.array([[4, 2], [2, 4], [2, 3], [3, 6], [4, 4], [9, 10], [6, 8], [9, 5], [8, 7], [10, 8]])
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])


# Initialize LDA with 1 component
lda = LinearDiscriminantAnalysis(n_components=1)


# Fit and transform the data
X_lda = lda.fit_transform(X, y)
print("Transformed X (LDA components):\n", X_lda)

ExtraTreesClassifier

In [ ]:
import numpy as np
from collections import Counter


class DecisionTree:
    """
    Single Decision Tree used in Extra Trees Classifier
    """
    def __init__(self, max_features=None, max_depth=None, min_samples_split=2):
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None


    def _best_split(self, X, y):
        """
        Finds a random split for the data by selecting random feature values.
        """
        m, n = X.shape
        if self.max_features is None:
            self.max_features = n


        best_feature, best_threshold = None, None
        best_gain = -1
        # Select a subset of features to examine
        features = np.random.choice(n, self.max_features, replace=False)


        for feature in features:
            # Randomly choose a threshold within the feature's range
            thresholds = np.random.uniform(min(X[:, feature]), max(X[:, feature]), 10)
            for threshold in thresholds:
                left_idx = X[:, feature] < threshold
                right_idx = X[:, feature] >= threshold


                # Calculate information gain
                if len(y[left_idx]) > 0 and len(y[right_idx]) > 0:
                    gain = self._information_gain(y, y[left_idx], y[right_idx])
                    if gain > best_gain:
                        best_gain, best_feature, best_threshold = gain, feature, threshold


        return best_feature, best_threshold


    def _information_gain(self, parent, left_child, right_child):
        """
        Calculates the information gain of a split.
        """
        weight_left = len(left_child) / len(parent)
        weight_right = 1 - weight_left
        return self._gini(parent) - (weight_left * self._gini(left_child) + weight_right * self._gini(right_child))


    def _gini(self, y):
        """
        Computes the Gini impurity for an array of classes.
        """
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - np.sum(ps ** 2)


    def _build_tree(self, X, y, depth=0):
        """
        Recursively builds the decision tree.
        """
        n_samples, n_features = X.shape
        num_labels = len(np.unique(y))


        # Stopping criteria
        if depth >= self.max_depth or num_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return leaf_value


        feature, threshold = self._best_split(X, y)
        if feature is None:
            return self._most_common_label(y)


        left_idx = X[:, feature] < threshold
        right_idx = X[:, feature] >= threshold
        left_subtree = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_subtree = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        return (feature, threshold, left_subtree, right_subtree)


    def _most_common_label(self, y):
        """
        Returns the most common label in an array.
        """
        counter = Counter(y)
        return counter.most_common(1)[0][0]


    def fit(self, X, y):
        self.tree = self._build_tree(X, y)


    def _predict(self, inputs, node):
        """
        Predicts a single data point by traversing the tree.
        """
        if not isinstance(node, tuple):
            return node


        feature, threshold, left, right = node
        if inputs[feature] < threshold:
            return self._predict(inputs, left)
        else:
            return self._predict(inputs, right)


    def predict(self, X):
        """
        Predicts class labels for X.
        """
        return [self._predict(inputs, self.tree) for inputs in X]


class ExtraTreesClassifier:
    """
    Extra Trees Classifier
    """
    def __init__(self, n_estimators=10, max_features=None, max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []


    def fit(self, X, y):
        """
        Train Extra Trees Classifier
        """
        for _ in range(self.n_estimators):
            tree = DecisionTree(max_features=self.max_features, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            indices = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[indices], y[indices]
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)


    def predict(self, X):
        """
        Predict class labels for X
        """
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)  # Shape: (n_samples, n_estimators)
        y_pred = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]
        return np.array(y_pred)


# Example usage:
# Define a sample dataset
X = np.array([[4, 2], [2, 4], [2, 3], [3, 6], [4, 4], [9, 10], [6, 8], [9, 5], [8, 7], [10, 8]])
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])


# Initialize the Extra Trees Classifier
extra_trees = ExtraTreesClassifier(n_estimators=10, max_features=1, max_depth=10)


# Fit the model
extra_trees.fit(X, y)


# Predict the class of a sample
y_pred = extra_trees.predict(X)
print("Predictions:", y_pred)

BaggingClassifier

In [ ]:
import numpy as np
from collections import Counter


class DecisionTree:
    """
    Single Decision Tree used in Bagging Classifier
    """
    def __init__(self, max_features=None, max_depth=None, min_samples_split=2):
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None


    def _best_split(self, X, y):
        m, n = X.shape
        if self.max_features is None:
            self.max_features = n


        best_feature, best_threshold = None, None
        best_gain = -1
        # Select a subset of features to examine
        features = np.random.choice(n, self.max_features, replace=False)


        for feature in features:
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_idx = X[:, feature] < threshold
                right_idx = X[:, feature] >= threshold


                if len(y[left_idx]) > 0 and len(y[right_idx]) > 0:
                    gain = self._information_gain(y, y[left_idx], y[right_idx])
                    if gain > best_gain:
                        best_gain, best_feature, best_threshold = gain, feature, threshold


        return best_feature, best_threshold


    def _information_gain(self, parent, left_child, right_child):
        weight_left = len(left_child) / len(parent)
        weight_right = 1 - weight_left
        return self._gini(parent) - (weight_left * self._gini(left_child) + weight_right * self._gini(right_child))


    def _gini(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return 1 - np.sum(ps ** 2)


    def _build_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        num_labels = len(np.unique(y))


        if depth >= self.max_depth or num_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return leaf_value


        feature, threshold = self._best_split(X, y)
        if feature is None:
            return self._most_common_label(y)


        left_idx = X[:, feature] < threshold
        right_idx = X[:, feature] >= threshold
        left_subtree = self._build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_subtree = self._build_tree(X[right_idx, :], y[right_idx], depth + 1)
        return (feature, threshold, left_subtree, right_subtree)


    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]


    def fit(self, X, y):
        self.tree = self._build_tree(X, y)


    def _predict(self, inputs, node):
        if not isinstance(node, tuple):
            return node


        feature, threshold, left, right = node
        if inputs[feature] < threshold:
            return self._predict(inputs, left)
        else:
            return self._predict(inputs, right)


    def predict(self, X):
        return [self._predict(inputs, self.tree) for inputs in X]




class BaggingClassifier:
    """
    Bagging Classifier with Decision Tree as the base estimator
    """
    def __init__(self, n_estimators=10, max_features=None, max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []


    def fit(self, X, y):
        """
        Train the Bagging Classifier
        """
        for _ in range(self.n_estimators):
            tree = DecisionTree(max_features=self.max_features, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            indices = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[indices], y[indices]
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)


    def predict(self, X):
        """
        Predict class labels for X
        """
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)  # Shape: (n_samples, n_estimators)
        y_pred = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]
        return np.array(y_pred)




# Example usage:
# Define a sample dataset
X = np.array([[4, 2], [2, 4], [2, 3], [3, 6], [4, 4], [9, 10], [6, 8], [9, 5], [8, 7], [10, 8]])
y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])


# Initialize the Bagging Classifier
bagging_clf = BaggingClassifier(n_estimators=10, max_features=1, max_depth=10)


# Fit the model
bagging_clf.fit(X, y)


# Predict the class of a sample
y_pred = bagging_clf.predict(X)
print("Predictions:", y_pred)

GaussianNB


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report


class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.var = {}
        self.priors = {}


        # Calculate mean, variance, and prior probabilities for each class
        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = X_c.mean(axis=0)
            self.var[c] = X_c.var(axis=0)
            self.priors[c] = X_c.shape[0] / X.shape[0]


    def _gaussian_density(self, class_idx, x):
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


    def _classify(self, x):
        posteriors = []
        for c in self.classes:
            prior = np.log(self.priors[c])  # Prior probability
            class_conditional = np.sum(np.log(self._gaussian_density(c, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]


    def predict(self, X):
        return [self._classify(x) for x in X]


# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Train and test the model
gnb = GaussianNaiveBayes()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)


print("Gaussian Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

MultinomialNB


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report


class MultinomialNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_feature_count = {}
        self.class_counts = {}
        self.class_totals = {}
        self.alpha = 1  # Laplace smoothing factor


        # Calculate counts for each feature per class
        for c in self.classes:
            X_c = X[y == c]
            self.class_feature_count[c] = np.sum(X_c, axis=0) + self.alpha
            self.class_counts[c] = X_c.shape[0]
            self.class_totals[c] = self.class_feature_count[c].sum()


        self.class_priors = {c: np.log(self.class_counts[c] / X.shape[0]) for c in self.classes}


    def _classify(self, x):
        posteriors = []
        for c in self.classes:
            prior = self.class_priors[c]
            conditional = np.sum(x * np.log(self.class_feature_count[c] / self.class_totals[c]))
            posterior = prior + conditional
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]


    def predict(self, X):
        return [self._classify(x) for x in X]


# Load the 20 Newsgroups dataset and preprocess
newsgroups = fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(newsgroups.data)
X, y = X_vec.toarray(), newsgroups.target


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Train and test the model
mnb = MultinomialNaiveBayes()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)


print("Multinomial Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


ComplementNB

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report


class ComplementNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.alpha = 1  # Laplace smoothing factor
        self.class_feature_count = {}
        self.class_totals = {}


        # Calculate counts for each feature complement per class
        for c in self.classes:
            X_c = X[y != c]  # Use data from all other classes (complement)
            self.class_feature_count[c] = np.sum(X_c, axis=0) + self.alpha
            self.class_totals[c] = self.class_feature_count[c].sum()


        self.class_priors = {c: np.log((X[y != c].shape[0]) / X.shape[0]) for c in self.classes}


    def _classify(self, x):
        posteriors = []
        for c in self.classes:
            prior = self.class_priors[c]
            conditional = np.sum(x * np.log(self.class_feature_count[c] / self.class_totals[c]))
            posterior = prior - conditional  # Note: we use the complement, so we subtract
            posteriors.append(posterior)
        return self.classes[np.argmin(posteriors)]  # Pick the class with the smallest posterior


    def predict(self, X):
        return [self._classify(x) for x in X]


# Load the 20 Newsgroups dataset and preprocess
newsgroups = fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(newsgroups.data)
X, y = X_vec.toarray(), newsgroups.target


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Train and test the model
cnb = ComplementNaiveBayes()
cnb.fit(X_train, y_train)
y_pred = cnb.predict(X_test)


print("Complement Naive Bayes Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Quadratic Discriminant Analysis


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report


class QuadraticDiscriminantAnalysis:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.cov = {}
        self.priors = {}


        # Calculate mean, covariance matrix, and prior probabilities for each class
        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.cov[c] = np.cov(X_c, rowvar=False) + 1e-6 * np.eye(X.shape[1])  # Add regularization
            self.priors[c] = X_c.shape[0] / X.shape[0]


    def _pdf_multivariate(self, x, mean, cov):
        size = len(x)
        det = np.linalg.det(cov)
        norm_const = 1.0 / (np.power((2 * np.pi), size / 2) * np.sqrt(det))
        x_mu = x - mean
        inv = np.linalg.inv(cov)
        result = np.exp(-0.5 * np.dot(np.dot(x_mu.T, inv), x_mu))
        return norm_const * result


    def _classify(self, x):
        posteriors = []
        for c in self.classes:
            prior = np.log(self.priors[c])
            likelihood = np.log(self._pdf_multivariate(x, self.mean[c], self.cov[c]))
            posterior = prior + likelihood
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]


    def predict(self, X):
        return [self._classify(x) for x in X]


# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Train and test the model
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)
y_pred = qda.predict(X_test)


print("Quadratic Discriminant Analysis Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))